Jika pakai CPU gunakan kode dibawah ini

In [ ]:
#r "nuget: TorchSharp-cpu"

using TorchSharp;
using static TorchSharp.torch;
using static TorchSharp.torch.nn;
using static TorchSharp.torch.nn.functional;
using Microsoft.DotNet.Interactive.Formatting;
using static TorchSharp.TensorExtensionMethods;
using static TorchSharp.torch.distributions;
using Microsoft.DotNet.Interactive.Formatting;

var style = TensorStringStyle.Julia;

//untuk formatting saat show isi variabel tensor
Formatter.SetPreferredMimeTypesFor(typeof(torch.Tensor), "text/plain");
Formatter.Register<torch.Tensor>((torch.Tensor x) => x.ToString(style, newLine: "\n"));

Installed Packages TorchSharp-cpu, 0.96.6

Jika pakai GPU gunakan kode di bawah ini

In [ ]:
#r "nuget: TorchSharp-cuda-windows"

using TorchSharp;
using static TorchSharp.torch;
using static TorchSharp.torch.nn;
using static TorchSharp.torch.nn.functional;
using Microsoft.DotNet.Interactive.Formatting;
using static TorchSharp.TensorExtensionMethods;
using static TorchSharp.torch.distributions;
using Microsoft.DotNet.Interactive.Formatting;

var style = TensorStringStyle.Julia;

Formatter.SetPreferredMimeTypesFor(typeof(torch.Tensor), "text/plain");
Formatter.Register<torch.Tensor>((torch.Tensor x) => x.ToString(style, newLine: "\n"));

## Modelling
Ini adalah fungsi utama dari library torchsharp, yaitu membuat model machine/deep learning. 

In [ ]:
public class NeuralModel : Module
    {
        //bikin NeuralNetwork dengan 3 layer : input, hidden dan output
        private Module InputLayer = Linear(4, 64); //4 node input
        private Module HiddenLayer = Linear(64, 64); // 64 node hidden layer
        private Module OutputLayer = Linear(64, 3); //3 node output
        //relu salah satu jenis activation function, cek disini: https://pytorch.org/docs/stable/nn.functional.html
        private Module relu1 = ReLU();
        //untuk menghindari terjadinya overfitting, bisa juga dengan regularization (weight_decay)
        private Module dropout1 = Dropout(0.25);

        public NeuralModel(torch.Device device = null) : base("NeuralModel")
        {
            RegisterComponents();
            //jika menggunakan GPU maka switch model menggunakan CUDA
            if (device != null && device.type == DeviceType.CUDA)
                this.to(device);
        }

        public override Tensor forward(Tensor input)
        {
            var layer1 = InputLayer.forward(input);
          
            var layer2 = HiddenLayer.forward(layer1);
            
            var layerRelu = relu1.forward(layer2);

            var layerDropOut = dropout1.forward(layerRelu);

            var layer3 = OutputLayer.forward(layerDropOut);           

            return layer3;
        }
    }

    //fungsi training

    void TrainModel(Module model, Tensor Input, Tensor Output, int Epoch=100){
        //fungsi loss function, untuk menghitung error antara input dan output. menggunakan mean-squared-error. ref:https://neptune.ai/blog/pytorch-loss-functions
        var loss = nn.functional.mse_loss();
        //learning rate, semakin besar semakin cepat mereduksi error, semakin kecil semakin detail dan lama proses trainingnya
        var learning_rate = 0.01f;
        //fungsi optimizer untuk meminimisasi error, https://pytorch.org/docs/stable/optim.html
        var optimizer = torch.optim.SGD(model.parameters(), learning_rate);

        // iterasi sebanyak epoch
        for (int i = 0; i < Epoch; i++) {
            using (var d = torch.NewDisposeScope())
            {
                // hitung loss
                using var output = loss(model.forward(Input), Output);

                // reset gradien sebelum lakukan back-propagation
                model.zero_grad();

                // lakukan back-progatation, untuk kalkulasi gradients dan update weight
                output.backward();

                optimizer.step();
            }
        }

    }


    //dengan GPU
    //var model = new NeuralModel(torch.CUDA);

    //dengan CPU
    var model = new NeuralModel();
    
    //data sample untuk input (features) dan output (label)
    var featuresTensor = rand(100,4);
    //featuresTensor.print();

    var labelTensor = rand(100,3);
    //labelTensor.print();

    //train model
    TrainModel(model,featuresTensor,labelTensor,100);

    //hitung akurasi, antara hasil prediksi model dan label data training    
    void HitungAkurasi(Module modelNN, Tensor Features, Tensor Labels){
    var refMax = Labels.argmax(1);
        Console.WriteLine("Referensi dari Label:");
        refMax.print();

        var predMax = modelNN.forward(Features).argmax(1);
        Console.WriteLine("Hasil Prediksi model:");
        predMax.print();

        var akurasi = (refMax == predMax).sum() / refMax.numel();
        Console.WriteLine($"Akurasi: { (akurasi * 100).item<float>() } %");
    }

    HitungAkurasi(model,  featuresTensor, labelTensor);

    //save model ke file (berisi weight dari network saja)
    model.save("weight_model.bin");

    //load model dari file
    var model1 = new NeuralModel();
    model1.load("weight_model.bin");
    Console.WriteLine("model disimpan, lalu di load kembali. Cek akurasi:");
    HitungAkurasi(model1,  featuresTensor, labelTensor);


Referensi dari Label:
[100], type = Int64, device = cpu
 2 1 2 1 1 0 2 1 1 0 1 2 0 2 0 2 2 1 0 2 0 1 1 1 0 2 1 2 0 0 2 2 1 0 0 1 1 0 0 1 2 2 2 1 0 2 0 0 ...

Hasil Prediksi model:
[100], type = Int64, device = cpu
 2 0 2 1 2 0 1 2 1 0 2 2 1 1 2 0 2 0 1 1 0 0 0 1 0 0 2 1 2 2 2 2 1 2 0 0 2 2 0 0 2 0 2 0 1 1 0 2 ...

Akurasi: 45 %
model disimpan, lalu di load kembali. Cek akurasi:
Referensi dari Label:
[100], type = Int64, device = cpu
 2 1 2 1 1 0 2 1 1 0 1 2 0 2 0 2 2 1 0 2 0 1 1 1 0 2 1 2 0 0 2 2 1 0 0 1 1 0 0 1 2 2 2 1 0 2 0 0 ...

Hasil Prediksi model:
[100], type = Int64, device = cpu
 1 2 2 1 2 0 1 1 1 0 0 2 1 2 2 0 0 0 0 0 0 0 0 0 2 1 2 0 2 2 2 0 0 2 0 2 2 1 0 0 2 1 2 0 1 1 0 2 ...

Akurasi: 46 %


In [ ]:
public class NeuralModel2 : Module
    {
        //bikin NeuralNetwork dengan 3 layer : input, hidden dan output
        private Module InputLayer = Linear(4, 64); //4 node input
        private Module HiddenLayer = Linear(64, 64); // 64 node hidden layer
        private Module OutputLayer = Linear(64, 3); //3 node output
        //relu salah satu jenis activation function, cek disini: https://pytorch.org/docs/stable/nn.functional.html
        private Module relu1 = ReLU();
        //untuk menghindari terjadinya overfitting, bisa juga dengan regularization (weight_decay)
        private Module dropout1 = Dropout(0.25);

        public NeuralModel2(torch.Device device = null) : base("NeuralModel")
        {
            RegisterComponents();
            //jika menggunakan GPU maka switch model menggunakan CUDA
            if (device != null && device.type == DeviceType.CUDA)
                this.to(device);
        }

        public override Tensor forward(Tensor input){
            //menggunakan sequential, membuat model yang berurutan dari layer kiri ke kanan
            var seq = nn.Sequential(("input",InputLayer),("hidden",HiddenLayer),("relu",relu1),("dropout",dropout1),("output",OutputLayer));
            return seq.forward(input);
        }
    }

    //fungsi training dengan learning rate scheduler, 
    //untuk awal training learning rate bagusnya learning rate agak besar biar error cepet berkurang, semakin banyak epoch kita turunkan lr agar semakin detail

    void TrainModel(Module model, List<Tensor> Inputs, List<Tensor> Outputs, int Epoch=100){
        //fungsi loss function, untuk menghitung error antara input dan output. menggunakan mean-squared-error. ref:https://neptune.ai/blog/pytorch-loss-functions
        var loss = nn.functional.mse_loss();
        //learning rate, semakin besar semakin cepat mereduksi error, semakin kecil semakin detail dan lama proses trainingnya
        var learning_rate = 0.01f;
        //fungsi optimizer untuk meminimisasi error, https://pytorch.org/docs/stable/optim.html
        var optimizer = torch.optim.Adam(model.parameters(), learning_rate);
        //scheduler ubah learning rate setiap epoch
        var scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 25, 0.95);

        // iterasi sebanyak epoch
        for (int i = 0; i < Epoch; i++) {
            using (var d = torch.NewDisposeScope())
            {
                for(int batchCount =0; batchCount<Inputs.Count;batchCount++ ){
                    // hitung loss
                    using var output = loss(model.forward(Inputs[batchCount]), Outputs[batchCount]);

                    // reset gradien sebelum lakukan back-propagation
                    model.zero_grad();

                    // lakukan back-progatation, untuk kalkulasi gradients dan update weight
                    output.backward();

                    optimizer.step();
                }
                scheduler.step();
            }
        }

    }


    //dengan GPU
    //var model = new NeuralModel2(torch.CUDA);

    //dengan CPU
    var model = new NeuralModel2();
    
    //data sample untuk input (features) dan output (label) dengan batch 
    var featuresTensor = Enumerable.Range(0,16).Select(_ => rand(100,4)).ToList<torch.Tensor>();
    //featuresTensor.print();

    var labelTensor = Enumerable.Range(0,16).Select(_ => rand(100,3)).ToList<torch.Tensor>();
    //labelTensor.print();

    //train model
    TrainModel(model,featuresTensor[0],labelTensor[0],100);

    //hitung akurasi, antara hasil prediksi model dan label data training    
    void HitungAkurasi(Module modelNN, Tensor Features, Tensor Labels){
    var refMax = Labels.argmax(1);
        Console.WriteLine("Referensi dari Label:");
        refMax.print();

        var predMax = modelNN.forward(Features).argmax(1);
        Console.WriteLine("Hasil Prediksi model:");
        predMax.print();

        var akurasi = (refMax == predMax).sum() / refMax.numel();
        Console.WriteLine($"Akurasi: { (akurasi * 100).item<float>() } %");
    }

    HitungAkurasi(model,  featuresTensor[0], labelTensor[0]);

    //save model ke file (berisi weight dari network saja)
    model.save("weight_model2.bin");

    //load model dari file
    var model1 = new NeuralModel2();
    model1.load("weight_model2.bin");
    Console.WriteLine("model disimpan, lalu di load kembali. Cek akurasi:");
    HitungAkurasi(model1,  featuresTensor[0], labelTensor[0]);

Referensi dari Label:
[100], type = Int64, device = cpu
 2 2 0 1 1 0 2 0 2 0 1 1 2 2 2 0 1 2 1 2 0 1 0 0 0 0 2 0 1 2 2 1 0 2 1 2 1 2 1 1 1 0 1 1 0 2 0 1 ...

Hasil Prediksi model:
[100], type = Int64, device = cpu
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...

Akurasi: 33 %
model disimpan, lalu di load kembali. Cek akurasi:
Referensi dari Label:
[100], type = Int64, device = cpu
 2 2 0 1 1 0 2 0 2 0 1 1 2 2 2 0 1 2 1 2 0 1 0 0 0 0 2 0 1 2 2 1 0 2 1 2 1 2 1 1 1 0 1 1 0 2 0 1 ...

Hasil Prediksi model:
[100], type = Int64, device = cpu
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...

Akurasi: 33 %
